In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np


In [ ]:
# Load and prepare your data
def prepare_data():
    data = pd.read_csv('Quote-Equity-BEL-EQ-14-09-2024-to-14-09-2025.csv', skipinitialspace=True)
    
    # Clean column names
    data.columns = data.columns.str.strip()
    
    # Convert date and clean numeric columns
    data['Date'] = pd.to_datetime(data['Date'], format='%d-%b-%Y')
    
    # Remove commas from volume/value columns
    for col in ['VOLUME', 'VALUE', 'No of trades']:
        data[col] = data[col].str.replace(',', '').astype(float)
    
    # Sort by date ascending
    data = data.sort_values('Date').reset_index(drop=True)
    
    return data


In [ ]:
# Create interactive candlestick chart
def create_interactive_candlestick():
    df = prepare_data()
    
    # Create subplot with secondary y-axis for volume
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=('BEL Price', 'Volume'),
        row_width=[0.7, 0.3]
    )
    
    # Add candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=df['Date'],
            open=df['OPEN'],
            high=df['HIGH'],
            low=df['LOW'],
            close=df['close'],
            name='BEL',
            increasing_line_color='#00ff88',
            decreasing_line_color='#ff4444',
            increasing_fillcolor='#00ff88',
            decreasing_fillcolor='#ff4444'
        ),
        row=1, col=1
    )
    
    # Add volume bars
    fig.add_trace(
        go.Bar(
            x=df['Date'],
            y=df['VOLUME'],
            name='Volume',
            marker_color='rgba(158,202,225,0.8)',
            yaxis='y2'
        ),
        row=2, col=1
    )
    
    # Update layout for interactivity
    fig.update_layout(
        title='BEL Interactive Candlestick Chart',
        xaxis_title='Date',
        yaxis_title='Price (₹)',
        template='plotly_dark',  # Try 'plotly_white', 'seaborn', 'ggplot2'
        height=800,
        showlegend=True,
        xaxis_rangeslider_visible=False,  # Remove range slider
        
        # Add range selector buttons
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1M", step="month", stepmode="backward"),
                    dict(count=3, label="3M", step="month", stepmode="backward"),
                    dict(count=6, label="6M", step="month", stepmode="backward"),
                    dict(step="all", label="All")
                ])
            ),
            rangeslider=dict(visible=False),
            type="date"
        )
    )
    
    # Update y-axes
    fig.update_yaxes(title_text="Price (₹)", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    
    return fig

In [4]:
# Create and show the chart
fig = create_interactive_candlestick()
fig.show()


In [6]:
def create_advanced_interactive_chart():
    df = prepare_data()
    
    # Calculate technical indicators
    df['SMA_20'] = df['close'].rolling(window=20).mean()
    df['SMA_50'] = df['close'].rolling(window=50).mean()
    
    # Calculate RSI
    def calculate_rsi(prices, window=14):
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))
    
    df['RSI'] = calculate_rsi(df['close'])
    
    # Create subplots
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=('Price & Moving Averages', 'Volume', 'RSI'),
        row_heights=[0.6, 0.2, 0.2]
    )
    
    # Candlestick
    fig.add_trace(
        go.Candlestick(
            x=df['Date'],
            open=df['OPEN'],
            high=df['HIGH'],
            low=df['LOW'],
            close=df['close'],
            name='BEL',
            yaxis='y1'
        ),
        row=1, col=1
    )
    
    # Moving averages
    fig.add_trace(
        go.Scatter(x=df['Date'], y=df['SMA_20'], name='SMA 20', 
                  line=dict(color='orange', width=1)),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=df['Date'], y=df['SMA_50'], name='SMA 50',
                  line=dict(color='blue', width=1)),
        row=1, col=1
    )
    
    # Volume
    colors = ['green' if close > open else 'red' 
              for close, open in zip(df['close'], df['OPEN'])]
    
    fig.add_trace(
        go.Bar(x=df['Date'], y=df['VOLUME'], name='Volume',
               marker_color=colors, opacity=0.7),
        row=2, col=1
    )
    
    # RSI
    fig.add_trace(
        go.Scatter(x=df['Date'], y=df['RSI'], name='RSI',
                  line=dict(color='purple', width=1)),
        row=3, col=1
    )
    
    # Add RSI reference lines
    fig.add_hline(y=70, line_dash="dash", line_color="red", opacity=0.5, row=3, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", opacity=0.5, row=3, col=1)
    
    # Update layout
    fig.update_layout(
        title='BEL Advanced Interactive Analysis',
        height=900,
        template='plotly_dark',
        showlegend=True,
        xaxis3_title='Date',
        
        # Hover mode
        hovermode='x unified',
        
        # Range selector
        xaxis3=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=30, label="30D", step="day", stepmode="backward"),
                    dict(count=90, label="3M", step="day", stepmode="backward"),
                    dict(count=180, label="6M", step="day", stepmode="backward"),
                    dict(step="all", label="1Y")
                ])
            ),
            rangeslider=dict(visible=True, thickness=0.05),
            type="date"
        )
    )
    
    # Update y-axes
    fig.update_yaxes(title_text="Price (₹)", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="RSI", range=[0, 100], row=3, col=1)
    
    return fig

In [7]:
# Create advanced chart
advanced_fig = create_advanced_interactive_chart()
advanced_fig.show()

In [9]:
# Save as HTML for sharing
advanced_fig.write_html("bel_interactive_candlestick.html")